In [ ]:
# Data handling
import pandas as pd
import numpy as np

# Backtrader framework
import backtrader as bt

# Logging
import logging

# Typing & datetime
from typing import List, Dict, Any, Optional
from datetime import datetime


# Create timestamped filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = f"backtest_{timestamp}.log"

# Get the root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Remove existing handlers (like StreamHandler added by Jupyter)
for h in logger.handlers[:]:
    logger.removeHandler(h)

# Add FileHandler
file_handler = logging.FileHandler(log_filename, mode='w')
formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Now logging goes only to file
logger.info("This will go only to the log file.")


In [ ]:
def load_csv_data(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path, parse_dates=["DateTime"])
    df.set_index("DateTime", inplace=True)

    # Ensure Crossover, SL, TP exist
    for col in ["Crossover", "SL", "TP"]:
        if col not in df.columns:
            df[col] = 0
    return df

In [ ]:
class CSVData(bt.feeds.PandasData):
    lines = ('Crossover', 'SL', 'TP')  # add Entry
    params = (
        ('datetime', None),
        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', -1),
        ('openinterest', -1),
        ('Crossover', -1),
        ('SL', -1),
        ('TP', -1),
    )


In [ ]:
class TradeLogger:
    """Centralized trade logging"""
    def __init__(self):
        self.trades: List[Dict[str, Any]] = []

    def log_trade(
        self,
        trade_id: int,
        symbol: str,
        strategy_name: str,
        entry_time: datetime,
        exit_time: datetime,
        entry_price: float,
        exit_price: float,
        position_size: int,
        direction: str,
        pnl: float,
        entry_index: int,
        exit_index: int,
        stop_loss: Optional[float],
        take_profit: Optional[float],
        commission: float,
        spread: float
    ) -> None:
        self.trades.append({
            "Trade ID": trade_id,
            "Symbol": symbol,
            "Strategy Name": strategy_name,
            "Entry Time": entry_time,
            "Exit Time": exit_time,
            "Entry Price": entry_price,
            "Exit Price": exit_price,
            "Position Size": position_size,
            "Direction": direction,
            "PnL": pnl,
            "PnL %": (pnl / entry_price * 100) if entry_price else None,
            "Candle Index Entry": entry_index,
            "Candle Index Exit": exit_index,
            "SL": stop_loss,
            "TP": take_profit,
            "Risk-Reward Ratio": ((take_profit - entry_price) / (entry_price - stop_loss)
                                  if stop_loss and take_profit and entry_price > stop_loss else None),
            "Commissions": commission,
            "Spread": spread
        })

    def to_dataframe(self) -> pd.DataFrame:
        return pd.DataFrame(self.trades)


In [ ]:
class CrossSignalStrategy(bt.Strategy):
    params = (
        ("size", 1),
        ("valid_signal_values", {1}),
    )

    def __init__(self):
        self.cross_signal = self.datas[0].Crossover
        self.sl = self.datas[0].SL
        self.tp = self.datas[0].TP
        self.trade_logger = TradeLogger()
        self.last_entry = {}

    def next(self):
        if self.position:
            return

        signal = self.cross_signal[0]
        if signal not in self.params.valid_signal_values:
            return

        entry_price = self.data.close[0]
        stop_loss = self.sl[0]
        take_profit = self.tp[0]

        if not self._validate_trade(entry_price, stop_loss, take_profit):
            logger.warning(f"Invalid trade params: entry={entry_price}, SL={stop_loss}, TP={take_profit}")
            return

        # Store entry details
        self.last_entry = {
            "price": entry_price,
            "sl": stop_loss,
            "tp": take_profit,
            "datetime": self.data.datetime.datetime(0),
            "index": len(self) - 1
        }

        # Place bracket order
        self.buy_bracket(size=self.params.size, price=entry_price, stopprice=stop_loss, limitprice=take_profit)
        logger.info(f"Placed order: price={entry_price}, SL={stop_loss}, TP={take_profit}")

    def notify_trade(self, trade):
        if trade.isclosed:
            self.trade_logger.log_trade(
                trade_id=len(self.trade_logger.trades)+1,
                symbol=self.data._name,
                strategy_name=type(self).__name__,
                entry_time=self.last_entry.get("datetime"),
                exit_time=self.data.datetime.datetime(0),
                entry_price=self.last_entry.get("price"),
                exit_price=trade.price,
                position_size=trade.size,
                direction="Long",
                pnl=trade.pnl,
                entry_index=self.last_entry.get("index"),
                exit_index=len(self) - 1,
                stop_loss=self.last_entry.get("sl"),
                take_profit=self.last_entry.get("tp"),
                commission=self.broker.getcommissioninfo(self.data).getcommission(trade.size, trade.price),
                spread=self.data.high[0] - self.data.low[0]
            )

    def stop(self):
        df = self.trade_logger.to_dataframe()
        if df.empty:
            logger.info("No trades executed.")
        else:
            logger.info(f"Total trades executed: {len(df)}")
            # Optional: save to CSV
            # df.to_csv("trades_log.csv", index=False)

    def _validate_trade(self, price, sl, tp):
        return all(isinstance(x, (int, float)) for x in [price, sl, tp]) and price > sl and price < tp


In [ ]:
# %% [5] Run Backtest with Extended Performance Summary
def run_backtest(csv_file: str, instrumentName="tst", cash=100000, size=1, valid_signals={1}):
    df = load_csv_data(csv_file)
    data = CSVData(dataname=df, name=instrumentName)

    cerebro = bt.Cerebro()
    cerebro.adddata(data)
    cerebro.addstrategy(CrossSignalStrategy, size=size, valid_signal_values=valid_signals)
    cerebro.broker.setcash(cash)
    cerebro.broker.setcommission(commission=0.001)

    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')

    # Run backtest
    results = cerebro.run()
    strat = results[0]

    # Trades DataFrame
    trades_df = strat.trade_logger.to_dataframe()

    # ==== Basic Stats ====
    trade_analyzer = strat.analyzers.trade_analyzer.get_analysis()
    total_trades = trade_analyzer.total.closed if hasattr(trade_analyzer.total, 'closed') else 0
    wins = trade_analyzer.won.total if hasattr(trade_analyzer.won, 'total') else 0
    losses = trade_analyzer.lost.total if hasattr(trade_analyzer.lost, 'total') else 0
    win_rate = (wins / total_trades * 100) if total_trades > 0 else 0

    total_pnl = trades_df['PnL'].sum() if not trades_df.empty else 0
    avg_pnl = trades_df['PnL'].mean() if not trades_df.empty else 0
    avg_win = trades_df[trades_df['PnL'] > 0]['PnL'].mean() if not trades_df.empty else 0
    avg_loss = trades_df[trades_df['PnL'] <= 0]['PnL'].mean() if not trades_df.empty else 0
    avg_rr = trades_df['Risk-Reward Ratio'].mean() if 'Risk-Reward Ratio' in trades_df.columns else None

    # Drawdown
    drawdown = strat.analyzers.drawdown.get_analysis()
    max_dd = drawdown.max.drawdown if hasattr(drawdown.max, 'drawdown') else 0

    # Sharpe
    sharpe = strat.analyzers.sharpe.get_analysis().get('sharperatio', None)

    # Starting & Ending cash
    starting_cash = cash
    ending_cash = cerebro.broker.getvalue()
    total_return = (ending_cash - starting_cash) / starting_cash * 100

    # Compile summary
    summary = {
        "Starting Cash": starting_cash,
        "Ending Cash": ending_cash,
        "Total Return %": total_return,
        "Total Trades": total_trades,
        "Wins": wins,
        "Losses": losses,
        "Win Rate %": win_rate,
        "Total PnL": total_pnl,
        "Average PnL per Trade": avg_pnl,
        "Average Winning Trade": avg_win,
        "Average Losing Trade": avg_loss,
        "Average Risk-Reward Ratio": avg_rr,
        "Max Drawdown %": max_dd,
        "Sharpe Ratio": sharpe
    }
    return trades_df, df, cerebro, summary


In [ ]:
# Run backtest
trades_df, data, cerebro, summary = run_backtest("applyiedMacd-1.csv")

summary

# trades_df.head()

In [ ]:
trades_df.to_csv("bt.result.csv")